In [4]:
from pymongo import MongoClient
import numpy as np
import json
import gensim
client = MongoClient("mongodb://localhost:27017")
db = client['chpreprcs_db']
coll = db['chpreprcs_coll']
print(gensim.__version__)
print(np.__version__)
print(json.__version__)

0.13.1
1.11.1
2.0.9


In [5]:
k = 1395958  # k = 1 ~ 1395958
_COLL_NUM_ = 1395958
_VEC_SIZE_ = 100
_SYN_VEC_SIZE_ = 30
doc = coll.find_one({"_id": k})['syntax_label']
print(doc)

['D', 'D', 'VC', 'Na', 'Na', 'DE', 'Na', 'PERIODCATEGORY']


In [6]:
class MySentences(object):
    def __init__(self, collection):
        self.coll = collection
    def __iter__(self):
        for k in range(1, _COLL_NUM_ + 1):
            doc = self.coll.find_one({"_id": k})
            s = doc['sentence']
            if len(s) > 5:
                yield s
class MySyntax(object):
    def __init__(self, collection):
        self.coll = collection
    def __iter__(self):
        for k in range(1, _COLL_NUM_ + 1):
            doc = self.coll.find_one({"_id": k})
            s = doc['syntax_label']
            if len(s) > 5:
                yield s
sentences = MySentences(coll)
syntax = MySyntax(coll)

In [ ]:
model = gensim.models.Word2Vec(sentences, window = 5, min_count = 10, _VEC_SIZE_ = 100, iter = 30)
model.save('w2v_chtag_35')
model_syntax = gensim.models.Word2Vec(syntax, window = 3, min_count = 10, size = _SYN_VEC_SIZE_, iter = 30)
model_syntax.save('w2v_chtag_syntax_35')

In [68]:
k = model.vocab.keys()
k[0]

TypeError: 'dict_keys' object does not support indexing

In [4]:
model.most_similar(['張藝謀'])

[('楊德昌', 0.5849970579147339),
 ('影壇', 0.5447032451629639),
 ('導演', 0.5381878614425659),
 ('電影', 0.5376237630844116),
 ('侯孝賢', 0.5223599076271057),
 ('動作片', 0.4982476234436035),
 ('好萊塢', 0.491036981344223),
 ('執導', 0.48953455686569214),
 ('張作驥', 0.48481354117393494),
 ('關錦鵬', 0.4823378920555115)]

In [3]:
model = gensim.models.Word2Vec.load('w2v_chtag')
model_syntax = gensim.models.Word2Vec.load('w2v_chtag_syntax')

ImportError: No module named 'UserList'

In [45]:
import random
import numpy as np

def make_cloze(num):
    if num > 1000: num = 1000
        
    idx = []
    for i in range(num):
        idx.append(round(random.uniform(1, _COLL_NUM_)))
    idx = sorted(set(idx[: num]))
    
    sentences = []
    for i in idx:
        sentences.append(coll.find_one({"_id": i})['sentence'])
    options = []
    keys = list(model.vocab.keys())
    nKeys = len(keys)
    for s in sentences:
        l = len(s)
        idx = list(range(0, nKeys))
        random.shuffle(idx)
        candidate = [keys[i] for i in idx[:3]]
#         count = 0
#         while count < 10:
        i = random.randint(0, l - 1)
#             if len(s[i]) > 4 or count == 9:
        options.append((i, [s[i]] + candidate))
#                 break
#             count += 1            
    return sentences, options

def cal_score(sentences, options):
    n = len(sentences)
    s = 0.
    ref = 0.
    for i in range(n):
        b = cloze_test(sentences[i], options[i][0], options[i][1])
        if b.argmax() == 0: s += 1.
        if random.randint(0, 3) == 0: ref += 1.
    return s/float(n), ref/float(n)


def cloze_test(string, index, candidates):
    temp = string[:]
    del temp[index]
    ref_vec = np.zeros(_VEC_SIZE_)
    l = 0
    for w in temp:
        if w in model.vocab:
            ref_vec += model[w]
        l += 1        
#     ref_vec /= float(l)
#     ref_vec = ref_vec / np.linalg.norm(ref_vec)
    nc = len(candidates)
    
    arr = np.zeros(nc)
    for i in range(nc):
        if candidates[i] in model.vocab:
            cand_vec = model[candidates[i]]
#             arr[i] = (ref_vec * cand_vec).sum()
            arr[i] = (ref_vec * (cand_vec / np.linalg.norm(cand_vec))).sum()
#     l, r = arr[:, j].min(), arr[:, j].max()
#     if r > l:
#         arr[:, j] = (arr[:, j] - l) / (r - l)
    return arr



def make_cloze_with_syntax(num):
    if num > 1000: num = 1000
        
    idx = []
    for i in range(num):
        idx.append(round(random.uniform(1, _COLL_NUM_)))
    idx = sorted(set(idx[: num]))
    
    sentences = []
    syntaxes = []
    for i in idx:
        sentences.append(coll.find_one({"_id": i})['sentence'])
        syntaxes.append(coll.find_one({"_id": i})['syntax_label'])
    options = []
    keys = list(model.vocab.keys())
    nKeys = len(keys)
    for s in sentences:
        l = len(s)
        idx = list(range(0, nKeys))
        random.shuffle(idx)
        candidate = [keys[i] for i in idx[:3]]
#         count = 0
#         while count < 10:
        i = random.randint(0, l - 1)
#             if len(s[i]) > 4 or count == 9:
        options.append((i, [s[i]] + candidate))
#                 break
#             count += 1            
    return sentences, syntaxes, options

def cloze_test2(string, syntax, index, cand_words, cand_syntaxes):
    str_temp = string[:]
    syn_temp = syntax[:]
    del str_temp[index]
    del syn_temp[index]
    ref_vec = np.zeros(_VEC_SIZE_)
    l = 0
    for w in temp:
        if w in model.vocab:
            ref_vec += model[w]
        l += 1
            
#     ref_vec /= float(l)
#     ref_vec = ref_vec / np.linalg.norm(ref_vec)
    nc = len(candidates)
    
    arr = np.zeros(nc)
    syn_vec = np.zeros(_SYN_VEC_SIZE_)
    for i in range(nc):
        if candidates[i] in model.vocab:
            cand_vec = model[candidates[i]]
#             arr[i] = (ref_vec * cand_vec).sum()
            arr[i] = (ref_vec * (cand_vec / np.linalg.norm(cand_vec))).sum()
#     l, r = arr[:, j].min(), arr[:, j].max()
#     if r > l:
#         arr[:, j] = (arr[:, j] - l) / (r - l)
    return arr

In [39]:
sentences, options = make_cloze(10)
for i in range(len(options)):
    print(i+1)
    print('{}\n{}\n--------------------------'.format(sentences[i], options[i]))


1
['他們', '要', '其餘', '族人', '遠遠', '的', '相隔', '{{CD}}幾', '里', '路', ',']
(10, [',', '機組', '寂靜', '安定下來'])
--------------------------
2
['就', '點頭', '答應', '讓', '她', '回家', '。']
(5, ['回家', '放流', '耶誕', '朝聖'])
--------------------------
3
['除了', '對', '殺', '人', '者', '依法', '究辦', '外', ',']
(2, ['殺', '腸病毒', '執行長', '喜怒哀樂'])
--------------------------
4
['然後', '用', '彩筆', '畫出', '各', '種', '不同', '的', '風箏', ',']
(5, ['種', '飆出', '改', '玉春園'])
--------------------------
5
['簡直', '是', '物超所值', '。']
(1, ['是', '憂愁', '豪邁', '果皮'])
--------------------------
6
['惟', '經', '分類', '後', '大致', '可', '分成', '({{CD}})', '沿街', '步道式', '開放', '空間', '、', '({{CD}})', '通路式', '開放', '空間', '、', '({{CD}})', '廣場式', '開放', '空間', '等', '{{CD}}', '大', '種類', '。']
(13, ['({{CD}})', '濫採', '家電業', '先聲'])
--------------------------
7
['但是', '在', '行文', '時', ',']
(4, [',', '避', '潭', '{{CD}}元化'])
--------------------------
8
['湖人', '全', '場', '投籃', '命中率', '{{CD}}成{{CD}}', ',']
(0, ['湖人', '感想', '悼念', '許可'])
--------------------------
9
['也', '是', '在', '

In [55]:
for i in range(10):
    sentences, options = make_cloze(1000)
    score, ref = cal_score(sentences, options)
    print('{:.3f}, {:.3f}'.format(score, ref))

0.555, 0.259
0.570, 0.239
0.545, 0.237
0.547, 0.257
0.548, 0.224
0.540, 0.257
0.526, 0.237
0.551, 0.259
0.556, 0.247
0.541, 0.253


In [70]:
print(list(model_syntax.vocab.keys()))

['Dfa', 'Cab', 'Dk', 'DASHCATEGORY', 'I', 'DOTCATEGORY', 'ETCCATEGORY', 'VK', 'T', 'VE', 'Neqb', 'VJ', 'SPCHANGECATEGORY', 'EXCLAMATIONCATEGORY', 'PERIODCATEGORY', 'VB', 'V_2', 'Cba', 'COMMACATEGORY', 'Nh', 'P', 'DM', 'PARENTHESISCATEGORY', 'Nb', 'VA', 'VI', 'VC', 'A', 'FW', 'Nep', 'Caa', 'SEMICOLONCATEGORY', 'VG', 'Neqa', 'Na', 'VD', 'SHI', 'Nc', 'EXCLANATIONCATEGORY', 'QUESTIONCATEGORY', 'D', 'Cbb', 'Dfb', 'VF', 'DE', 'VAC', 'Di', 'Nd', 'Dj', 'COLONCATEGORY', 'Neu', 'VCL', 'Nf', 'VHC', 'Nv', 'Ncd', 'VL', 'VH', 'neu', 'Nes', 'Da', 'Ng', 'b', 'PAUSECATEGORY']


In [75]:
model_syntax.most_similar(['PAUSECATEGORY'])

[('Cab', 0.8692370653152466),
 ('Caa', 0.822738528251648),
 ('Nb', 0.6957693099975586),
 ('Na', 0.6932872533798218),
 ('Nc', 0.6587270498275757),
 ('ETCCATEGORY', 0.632373571395874),
 ('VA', 0.5940077900886536),
 ('VC', 0.5900530219078064),
 ('A', 0.5898308753967285),
 ('Nv', 0.491829514503479)]

In [21]:
class Word2Syn(object):
    def __init__(self, word_set, syn_set):
        import numpy as np
        import json
        self.word_set = word_set
        self.syn_set = syn_set

        lw = len(word_set)
        ls = len(syn_set)
        word_idx = {}
        for i in range(lw):
            word_idx[word_set[i]] = i
        with open('word_idx.json', 'w') as fp:
            json.dump(word_idx, fp)
        syn_idx = {}
        for i in range(ls):
            syn_idx[syn_set[i]] = i
        with open('syn_idx.json', 'w') as fp:
            json.dump(syn_idx, fp)
        self.word_idx = word_idx
        self.syn_idx = syn_idx
        self.word_syn_mapping = np.zeros((lw, ls))
    def generate(self, coll):
        import numpy as np
        for i in range(1, _COLL_NUM_ + 1):
            sen = coll.find_one({"_id": i})['sentence']
            syn = coll.find_one({"_id": i})['syntax_label']
            l = len(sen)
            for j in range(l):
                try:
                    if (sen[j] in self.word_idx) and (syn[j] in self.syn_idx):
                        self.word_syn_mapping[self.word_idx[sen[j]], self.syn_idx[syn[j]]] += 1
                except:
                    print(i)
                    print(j)
                    print(sen)
                    print(syn)
        np.save('w2s_mapping', self.word_syn_mapping)

w2s = Word2Syn(list(model.vocab), list(model_syntax.vocab))
w2s.generate(coll)
        

749464
10
['洋基隊', '終場', '{{CD}}', '', '比', '{{CD}}', '神奇', '演出', '大', '逆轉', ',']
['Nb', 'Na', 'Neu', 'Caa', 'Neu', 'VH', 'VC', 'VH', 'VA', 'COMMACATEGORY']
773498
16
['身', '置', '其中', '連', '坐', '在', '你', '對面', '人', '說話', '都', '', '很', '難', '聽', '清楚', ',']
['Na', 'VC', 'Nep', 'Cbb', 'VA', 'P', 'Nh', 'Ncd', 'Na', 'VA', 'D', 'Dfa', 'VH', 'VC', 'VH', 'COMMACATEGORY']
775038
20
['全球', '最', '大', '療傷', '藥品', '供應商', '{{FW}}', '&', '{{FW}}', '目前', '正', '行銷', '{{CD}}', '種', '含有', '奈米', '大小', '結晶銀', '的', '抗菌', '藥膏', ',']
['Nc', 'Dfa', 'VH', 'VA', 'Na', 'Na', 'Nb', 'Nd', 'D', 'VCL', 'Neu', 'Nf', 'VJ', 'Na', 'Na', 'Na', 'DE', 'VH', 'Na', 'COMMACATEGORY']
775038
21
['全球', '最', '大', '療傷', '藥品', '供應商', '{{FW}}', '&', '{{FW}}', '目前', '正', '行銷', '{{CD}}', '種', '含有', '奈米', '大小', '結晶銀', '的', '抗菌', '藥膏', ',']
['Nc', 'Dfa', 'VH', 'VA', 'Na', 'Na', 'Nb', 'Nd', 'D', 'VCL', 'Neu', 'Nf', 'VJ', 'Na', 'Na', 'Na', 'DE', 'VH', 'Na', 'COMMACATEGORY']
840658
7
['在', '{{FW}}', '{{FW}}', '溫暖', '的', '歌聲', '裡', ',']
['P',

In [28]:
i = 1059451
sen = coll.find_one({"_id": i})['sentence']
syn = coll.find_one({"_id": i})['syntax_label']
print(len(sen))
print(len(syn))


16
15


In [3]:
word_syn_mapping = np.load('w2s_mapping.npy')
with open('word_idx.json', 'r') as fp:
    word_idx = json.load(fp)
word = list(model.vocab)
i = 3333
print(word[i])
print(word_idx[word[i]])
word_syn_mapping[i, :]

NameError: name 'model' is not defined